In [6]:
cd module2-sql-for-analysis

/Users/connorheraty/Desktop/Repos/DS-Unit-3-Sprint-2-SQL-and-Databases/module2-sql-for-analysis


In [7]:
!pip install psycopg2-binary

### Import Dataset

In [53]:
import psycopg2
import pandas as pd

In [54]:
df = pd.read_csv('titanic.csv')

In [55]:
df.head(30)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


In [56]:
df['Name'] = df['Name'].str.replace("'", '`')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
Survived                   887 non-null int64
Pclass                     887 non-null int64
Name                       887 non-null object
Sex                        887 non-null object
Age                        887 non-null float64
Siblings/Spouses Aboard    887 non-null int64
Parents/Children Aboard    887 non-null int64
Fare                       887 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.5+ KB


### Connect to PostgreSQL

In [58]:
dbname = 'rbdyjzel'
user = 'rbdyjzel'
password = 'PqamaH9-y5MGtTxPP__tkNvls9Nj0WWQ' # Don't commit this!
host = 'raja.db.elephantsql.com'

In [59]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [60]:
pg_curs = pg_conn.cursor()

### Transform pandas dataframe to SQL

In [61]:
#df.to_sql(name='titanic', con=sl_conn, if_exists='replace')

In [62]:
import sqlite3
# Convert to SQLite3
sl_conn = sqlite3.connect('titanic.sqlite3')
df.to_sql(name='titanic', con=sl_conn, if_exists='replace')
sl_curs = sl_conn.cursor()

In [63]:
human = sl_curs.execute('SELECT * FROM titanic;').fetchall()

In [64]:
human[0]

(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [65]:
df.head(1)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.25


In [31]:
create_titanic_table = """
CREATE TABLE titanic (
  id SERIAL PRIMARY KEY,
  survived INT,
  pclass INT,
  name VARCHAR(100),
  sex VARCHAR(10),
  age INT,
  siblings_or_spouses_aboard INT,
  parents_children_aboard INT,
  fare FLOAT
)
"""

In [32]:
pg_curs.execute(create_titanic_table)

DuplicateTable: relation "titanic" already exists


In [50]:
attempted_insert = """
  INSERT INTO titanic
  VALUES""" + str(human[0])
attempted_insert

"\n  INSERT INTO titanic\n  VALUES(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)"

In [51]:
pg_curs.execute(attempted_insert)

UniqueViolation: duplicate key value violates unique constraint "titanic_pkey"
DETAIL:  Key (id)=(0) already exists.


In [68]:
pg_curs.execute('SELECT * FROM titanic;')
pg_human = pg_curs.fetchall()

In [69]:
human[28]

(28, 1, 3, 'Miss. Ellen O`Dwyer', 'female', 24.0, 0, 0, 7.8792)

In [70]:
pg_human[0]

(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22, 1, 0, 7.25)

In [71]:
for humans in human[1:]:
    insert_human = """
    INSERT INTO titanic
    VALUES """ + str(humans)
    pg_curs.execute(insert_human)
pg_conn.commit()

UniqueViolation: duplicate key value violates unique constraint "titanic_pkey"
DETAIL:  Key (id)=(1) already exists.


In [72]:
pg_human[-2]

(885, 1, 1, 'Mr. Karl Howell Behr', 'male', 26, 0, 0, 30.0)

In [75]:
for humans, pg_humans in zip(human, pg_human):
    assert human == pg_human

AssertionError: 